In [1]:
import json
import requests
import datetime as dt
import pickle
import pandas as pd

# An api key is emailed to you when you sign up to a plan
api_key = 'da3195c8356b3b0ee3e5a6682ed9c537'


In [2]:
teams = pd.read_csv('teams_table.csv')

In [14]:
sports_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
    'api_key': api_key,
    'sport': 'basketball_ncaab',
    'region': 'us',
    'oddsFormat': 'american'
})


# fbs_response = requests.get('https://api.fastbreakstats.com/teams')
# fbs_data = json.loads(fbs_response.text)

sports_json = json.loads(sports_response.text)
date = dt.datetime.today().date().strftime('%Y-%m-%d')
with open(str(date)+'.json', 'w') as outfile:
    json.dump(sports_json, outfile)
# with open('odds.json') as f:
#     sports_json = json.load(f)

In [3]:
date = dt.datetime.today().date().strftime('%Y-%m-%d')
with open(str(date)+'.json') as f:
    sports_json = json.load(f)

In [4]:
def getTeamData(name):
    name = name.replace(" ", "%20")
    baseUrl="https://api.fastbreakstats.com/teams/"
    params="?q=current"
    url=baseUrl+name+params

    r = requests.get(url = url).json()[0]
    bpival = r['BPIval']
    kpval = r['KenPomVal']
    srsval = r['SRS']
    return kpval, bpival, srsval

In [5]:
loaded_model = pickle.load(open('6yr_LR.sav', 'rb'))
loaded_model.coef_

array([[-0.14904683,  0.02408791, -0.00285878,  0.14617775, -0.05714497,
         0.03016929]])

In [6]:
def get_needed_odds1(conf):

    payout = round(((.1+(1-conf))/conf),4)
    if 100/payout > 100:
        odds = round(-100/payout)
    else:
        odds = round(100*payout)
    return odds

In [7]:
def get_ev1(odds,conf):
    if int(odds) > 0:
        payout = odds/100
    else:
        payout = -100/odds
        
    ev = (conf * payout) - (1-conf)
    return ev

In [8]:
for game in sports_json['data']:
    #Loop through and get odds
    for site in game['sites']:
        if site['site_key'] == 'bovada':
            odds = site['odds']['h2h']
            sitex = "bovada"
            break
        if site['site_key'] == 'draftkings':
            odds = site['odds']['h2h']
            sitex = "DK"
        if site['site_key'] == 'williamhill_us':
            odds = site['odds']['h2h']
            sitex = "WH"
            
    home = game['home_team']
    if game['home_team'] == game['teams'][0]:
        away = game['teams'][1]
        home_odds = odds[0]
        away_odds = odds[1]
    else:
        away = game['teams'][0]
        home_odds = odds[1]
        away_odds = odds[0]
    
    try:
        home = home.replace("-", " ")
        home = home.replace("(", "")
        home = home.replace(")", "")
        home = home.replace(".", "")
        away = away.replace("-", " ")
        away = away.replace("(", "")
        away = away.replace(")", "")
        away = away.replace(".", "")
        homekp, homebpi, homesrs = getTeamData(teams.loc[teams['odds_api_name'] == home]['kenpom_name'].values[0])
        awaykp, awaybpi, awaysrs = getTeamData(teams.loc[teams['odds_api_name'] == away]['kenpom_name'].values[0])
    except:
        print(game)
        pass
        
    print("{0} @ {1}".format(away,home))
    res = loaded_model.predict_proba([[float(awaykp), float(awaybpi), float(awaysrs), float(homekp), float(homebpi), float(homesrs)]])
#     print(res[0])
    print("{0} & {1} from {2}".format(away_odds,home_odds,sitex))
    away_needed_odds = get_needed_odds1(res[0][0])    
    home_needed_odds = get_needed_odds1(res[0][1])
    away_ev = get_ev1(away_odds,res[0][0])
    home_ev = get_ev1(home_odds,res[0][1])
    if away_odds > away_needed_odds:
        print("{0} is a good bet on {1} odds with an Expected Value of {2}".format(away,away_odds,away_ev))
    if home_odds > home_needed_odds:
        print("{0} is a good bet on {1} odds with an Expected Value of {2}".format(home,home_odds,home_ev))
    print()

St Bonaventure Bonnies @ Akron Zips
-225 & 185 from bovada

Houston Baptist Huskies @ Rice Owls
725 & -1299 from bovada

Chicago St Cougars @ Illinois St Redbirds
1600 & -7000 from WH

St Francis BKN Terriers @ Central Connecticut St Blue Devils
-155 & 135 from bovada

Stanford Cardinal @ CSU Northridge Matadors
-2000 & 950 from bovada

Buffalo Bulls @ Miami OH RedHawks
-160 & 140 from WH
Miami OH RedHawks is a good bet on 140 odds with an Expected Value of 0.10468160212827504

Coastal Carolina Chanticleers @ Wofford Terriers
180 & -220 from bovada
Wofford Terriers is a good bet on -220 odds with an Expected Value of 0.11767670986605111

Longwood Lancers @ Radford Highlanders
165 & -190 from bovada
Radford Highlanders is a good bet on -190 odds with an Expected Value of 0.10505683343582767

Clemson Tigers @ Virginia Tech Hokies
110 & -130 from bovada

Charlotte 49ers @ Davidson Wildcats
475 & -750 from bovada

Minnesota Golden Gophers @ Illinois Fighting Illini
300 & -400 from bovada

